# Configuration Usage Analyzer

This notebook helps you identify which files in a project use a specific configuration setting. It searches through your codebase to find references to particular configuration settings and visualizes the relationships between files and configurations.

## Import Required Libraries

Import libraries for file system operations, text processing, and data visualization.

In [ ]:
import os
import re
import argparse
from pathlib import Path
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go

## Parse Command Line Arguments

Create functions to parse command line arguments to specify the configuration to search for and the directory to search in.

In [ ]:
def parse_arguments():
    """
    Parse command line arguments when running as a script.
    Returns arguments that can be used within the notebook.
    """
    # If running in a notebook, we'll use these default values
    class Args:
        def __init__(self):
            self.config_pattern = None
            self.directory = None
            self.extensions = ['.py', '.js', '.ts', '.json', '.yaml', '.yml', '.toml', '.ini', '.conf']
            self.exclude_dirs = ['node_modules', 'venv', '.git', '__pycache__', 'build', 'dist']
            
    # Try to parse command line arguments if run as a script
    try:
        parser = argparse.ArgumentParser(description='Find files using specific configuration settings')
        parser.add_argument('--config', '-c', dest='config_pattern', required=True,
                            help='Configuration pattern to search for (e.g., "DEBUG_MODE")')
        parser.add_argument('--dir', '-d', dest='directory', default='.',
                            help='Directory to search in (default: current directory)')
        parser.add_argument('--ext', '-e', dest='extensions', nargs='+',
                            default=['.py', '.js', '.ts', '.json', '.yaml', '.yml', '.toml', '.ini', '.conf'],
                            help='File extensions to search (default: common code and config files)')
        parser.add_argument('--exclude', '-x', dest='exclude_dirs', nargs='+',
                            default=['node_modules', 'venv', '.git', '__pycache__', 'build', 'dist'],
                            help='Directories to exclude from search')
        
        args = parser.parse_args()
        return args
    except:
        # If running in notebook, return default args object
        print("Running in notebook mode. Use set_search_params() to configure search parameters.")
        return Args()

# Function to set parameters when running in notebook
def set_search_params(config_pattern, directory='.', 
                     extensions=['.py', '.js', '.ts', '.json', '.yaml', '.yml', '.toml', '.ini', '.conf'],
                     exclude_dirs=['node_modules', 'venv', '.git', '__pycache__', 'build', 'dist']):
    """Set the search parameters for the configuration search."""
    args = parse_arguments()
    args.config_pattern = config_pattern
    args.directory = directory
    args.extensions = extensions
    args.exclude_dirs = exclude_dirs
    return args

# Get default arguments
args = parse_arguments()

## Search for Configuration Usage

Implement functions to recursively search through files for specific configuration patterns or references.